In [1]:
from clustertools import build_datacube
from clustertools.parameterset import build_parameter_set

from pprint import pprint
import numpy as np
from train_monuseg_hard_clustertools import weight_exclude, exclude_target_and_dice_calibration
from train_monuseg_hard_clustertools import no_distillation_filter, filter_nc_rr, min_weight_only_for_entropy
from train_monuseg_baseline_clustertools import min_weight_for_entropy, not_using_incomplete
from ipnb_helpers import ExperimentReader, plot_table
from train_segpc_hard_clustertools import SegpcDatasetConstraint

In [2]:
import __main__
setattr(__main__, "weight_exclude", weight_exclude)
setattr(__main__, "exclude_target_and_dice_calibration", exclude_target_and_dice_calibration)
setattr(__main__, "no_distillation_filter", no_distillation_filter)
setattr(__main__, "filter_nc_rr", filter_nc_rr)
setattr(__main__, "min_weight_only_for_entropy", min_weight_only_for_entropy)
setattr(__main__, "min_weight_for_entropy", min_weight_for_entropy)
setattr(__main__, "not_using_incomplete", not_using_incomplete)
setattr(__main__, "SegpcDatasetConstraint", SegpcDatasetConstraint)

In [3]:
baseline = ExperimentReader("monuseg-unet-baseline", "monuseg-reeval-baseline", "rseed")
hard_exp = ExperimentReader("monuseg-unet-hard", "monuseg-reeval-hard", "monu_ms")


In [4]:
rows = [
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.05'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.2'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '0.5'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "constant", "weights_constant": '1.0'},
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "balance_gt" },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_entropy", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_entropy", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "hard_dice", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.05'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.2'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '0.5'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "constant", "weights_constant": '1.0'},
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "balance_gt" },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_entropy", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_entropy", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_consistency", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.1' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '1', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "absolute", "weights_minimum": '0.5' },
    {"no_distillation": 'False', "distil_target_mode": "soft", "weights_mode": "pred_merged", "weights_neighbourhood": '2', "weights_consistency_fn": "quadratic", "weights_minimum": '0.5' },
    {"no_distillation": 'True' },
]


In [21]:
hard_exp_fn = lambda **kwargs: hard_exp.get_reeval_metric("test_hard_dice", **kwargs)
columns = [
    (hard_exp_fn, {"monu_nc": '1', "monu_rr": '0.9', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '2', "monu_rr": '0.9', "n_calibration": '1'}),
    (hard_exp_fn, {"monu_nc": '2', "monu_rr": '0.5', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '3', "monu_rr": '0.5', "n_calibration": '1'}),
    (hard_exp_fn, {"monu_nc": '4', "monu_rr": '0.25', "n_calibration": '0'}),
    (hard_exp_fn, {"monu_nc": '5', "monu_rr": '0.25', "n_calibration": '1'})
]


In [22]:
plot_table(rows, columns, 30)


\begin{table*}
\begin{tabular}{|ccc|cccccc|}
\multicolumn{3}{|c|}{$|\mathcal{D}_l|/|\mathcal{D}_s|$} & 1/29 & 1/28 & 2/28 & 2/27 & 4/26 & 4/25\\
\multicolumn{3}{|c|}{$\rho$} &  90\% &  90\% &  50\% &  50\% &  25\% &  25\%\\
\multicolumn{3}{|c|}{$|\mathcal{D}_{cal}|$} & 0 & 1 & 0 & 1 & 0 & 1\\
\hline
\multicolumn{9}{|l|}{Constant ($C$)} \\
\hline
             \multicolumn{3}{|c|}{$1.0$}  &                     & 74.23 ± 2.43 &                    & 79.11 ± 0.85 &                    & 78.83 ± 0.98 \\
\hline
\multicolumn{9}{|l|}{Balance} \\
\hline
                                     & &  &                     & 77.56 ± 0.66 &                    & 78.80 ± 0.80 &                    & 78.45 ± 1.23 \\
\hline
\multicolumn{9}{|l|}{Entropy - $w_{\text{min}}$} \\
\hline
             \multicolumn{3}{|c|}{$0.1$}  &                     & 77.31 ± 1.09 &                    & 78.27 ± 1.22 &                    & 77.05 ± 2.79 \\
             \multicolumn{3}{|c|}{$0.5$}  &                     & 74.43 ± 2.4

In [11]:
hard_exp._cube.domain

{'distil_target_mode': ['hard_dice', 'soft'],
 'monu_ms': ['13315092',
  '21081788',
  '26735830',
  '35788921',
  '56755036',
  '56882282',
  '65682867',
  '91090292',
  '93410762',
  '96319575'],
 'monu_nc': ['1', '2', '3', '4', '5'],
 'monu_rr': ['0.25', '0.5', '0.9'],
 'n_calibration': ['0', '1'],
 'no_distillation': ['False', 'True'],
 'weights_minimum': ['0.0', '0.1', '0.5'],
 'weights_mode': ['balance_gt',
  'constant',
  'pred_consistency',
  'pred_entropy',
  'pred_merged'],
 'weights_neighbourhood': ['1', '2']}

In [20]:
hard_exp._cube(
    monu_nc="4", 
    n_calibration="0", 
    monu_rr="0.5", 
    no_distillation="False", 
    weights_mode="pred_merged", 
    weights_neighbourhood="2",
    distil_target_mode='soft',
    weights_minimum="0.0").diagnose()

{'Missing ratio': 1.0,
 'At least one missing': {'monu_ms': ['13315092',
   '21081788',
   '26735830',
   '35788921',
   '56755036',
   '56882282',
   '65682867',
   '91090292',
   '93410762',
   '96319575']},
 'All there': {'monu_ms': []}}

In [29]:
bl = baseline.get_reeval_metric("test_hard_dice", **{})
bl_avg, bl_std = np.mean(bl, axis=0), np.std(bl, axis=0)
bl_best = np.argmax(bl_avg)
print("{:2.2f} ± {:2.2f}".format(bl_avg[bl_best] * 100, bl_std[bl_best] * 100))

79.59 ± 0.53
